# --- Day 22: Monkey Market ---

On the Market, the buyers seem to use random prices, but their prices are actually only pseudorandom! If you know the secret of how they pick their prices, you can wait for the perfect time to sell.

The part about secrets is literal, the Historian explains. Each buyer produces a pseudorandom sequence of secret numbers where each secret is derived from the previous.

In particular, each buyer's secret number evolves into the next secret number in the sequence via the following process:

- Calculate the result of multiplying the secret number by 64. Then, mix this result into the secret number. Finally, prune the secret number.
- Calculate the result of dividing the secret number by 32. Round the result down to the nearest integer. Then, mix this result into the secret number. Finally, prune the secret number.
- Calculate the result of multiplying the secret number by 2048. Then, mix this result into the secret number. Finally, prune the secret number.

Each step of the above process involves mixing and pruning:

- To **mix** a value into the secret number, calculate the bitwise XOR of the given value and the secret number. Then, the secret number becomes the result of that operation. (If the secret number is 42 and you were to mix 15 into the secret number, the secret number would become 37.)
- To **prune** the secret number, calculate the value of the secret number modulo 16777216. Then, the secret number becomes the result of that operation. (If the secret number is 100000000 and you were to prune the secret number, the secret number would become 16113920.)
After this process completes, the buyer is left with the next secret number in the sequence. The buyer can repeat this process as many times as necessary to produce more secret numbers.

So, if a buyer had a secret number of 123, that buyer's next ten secret numbers would be:
```
15887950
16495136
527345
704524
1553684
12683156
11100544
12249484
7753432
5908254
```
Each buyer uses their own secret number when choosing their price, so it's important to be able to predict the sequence of secret numbers for each buyer. Fortunately, the Historian's research has uncovered the initial secret number of each buyer (your puzzle input). For example:
```
1
10
100
2024
```
This list describes the initial secret number of four different secret-hiding-spot-buyers on the Monkey Exchange Market. If you can simulate secret numbers from each buyer, you'll be able to predict all of their future prices.

In a single day, buyers each have time to generate 2000 new secret numbers. In this example, for each buyer, their initial secret number and the 2000th new secret number they would generate are:
```
1: 8685429
10: 4700978
100: 15273692
2024: 8667524
```
Adding up the 2000th new secret number for each buyer produces 37327623.

For each buyer, simulate the creation of 2000 new secret numbers. What is the sum of the 2000th secret number generated by each buyer?

In [ ]:
from fastcore.utils import L
from aocd import get_data

inp = get_data(day=22, year=2024)

samp = """1
10
100
2024"""
print(samp)

1
10
100
2024


Summarize the task for me.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠

This is Advent of Code 2024 Day 22. The task is:

1. Each buyer has a secret number that evolves through a specific process
2. The evolution involves three steps, each using "mix" (XOR) and "prune" (mod 16777216):
   - Multiply by 64, mix, prune
   - Divide by 32 (floor), mix, prune
   - Multiply by 2048, mix, prune
3. Given initial secret numbers, simulate 2000 iterations for each buyer
4. Sum up all the 2000th secret numbers

Want me to help you work through the implementation step by step, or would you prefer to tackle it yourself first?

In [ ]:
42 ^ 15

37

In [ ]:
(1 * 64) ^ 1

65

In [ ]:
def mix(secret, num):
    return secret ^ num
mix(15, 42)

37

In [ ]:
def prune(num):
    return num % 16777216
prune(100000000)

16113920

In [ ]:
def calc(secret):
    # first step
    res = secret * 64
    secret = prune(mix(secret, res))
    # second step 
    res = secret // 32
    secret = prune(mix(secret, res))
    # third step
    res = secret * 2048
    return prune(mix(secret, res))
calc(123)

15887950

In [ ]:
calc(15887950)

16495136

In [ ]:
calc(16495136)

527345

How can I turn calc into a generator function?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Add `while True:` around the body and `yield secret` at the end (or start). Something like:

```python
def calc_gen(secret):
    while True:
        secret = secret * 64
        secret = prune(mix(secret, secret // 64))
        # ... rest of steps
        yield secret
```

Or if you want to keep your existing `calc` function, you could wrap it:

```python
def calc_gen(secret):
    while True:
        secret = calc(secret)
        yield secret
```

Which approach appeals to you?

In [ ]:
def calc_gen(secret):
    while True:
        secret = calc(secret)
        yield secret

In [ ]:
from itertools import islice
list(islice(calc_gen(123), 10))

[15887950,
 16495136,
 527345,
 704524,
 1553684,
 12683156,
 11100544,
 12249484,
 7753432,
 5908254]

How can I get nth number from the generator?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠

You're already close! Use `islice` to skip to the nth element:

```python
next(islice(calc_gen(123), 1999, None))  # Gets the 2000th value
```

Or you could index into your list approach if you only need one value occasionally.

In [ ]:
next(islice(calc_gen(1), 1999, None))  # Gets the 2000th value

8685429

In [ ]:
def get_2000th(init_secret):
    return next(islice(calc_gen(init_secret), 1999, None))  # Gets the 2000th value
get_2000th(1)

8685429

In [ ]:
xs = L(samp.splitlines()).map(int)
xs

[1, 10, 100, 2024]

In [ ]:
xs.map(get_2000th)

[8685429, 4700978, 15273692, 8667524]

In [ ]:
xs.map(get_2000th).sum()

37327623

In [ ]:
def solve(inp):
    xs = L(inp.splitlines()).map(int)
    return xs.map(get_2000th).sum()
solve(inp)

20068964552

# --- Part Two ---
Of course, the secret numbers aren't the prices each buyer is offering! That would be ridiculous. Instead, the prices the buyer offers are just the ones digit of each of their secret numbers.

So, if a buyer starts with a secret number of 123, that buyer's first ten prices would be:
```
3 (from 123)
0 (from 15887950)
6 (from 16495136)
5 (etc.)
4
4
6
4
4
2
```
This price is the number of bananas that buyer is offering in exchange for your information about a new hiding spot. However, you still don't speak monkey, so you can't negotiate with the buyers directly. The Historian speaks a little, but not enough to negotiate; instead, he can ask another monkey to negotiate on your behalf.

Unfortunately, the monkey only knows how to decide when to sell by looking at the changes in price. Specifically, the monkey will only look for a specific sequence of four consecutive changes in price, then immediately sell when it sees that sequence.

So, if a buyer starts with a secret number of 123, that buyer's first ten secret numbers, prices, and the associated changes would be:

```
     123: 3 
15887950: 0 (-3)
16495136: 6 (6)
  527345: 5 (-1)
  704524: 4 (-1)
 1553684: 4 (0)
12683156: 6 (2)
11100544: 4 (-2)
12249484: 4 (0)
 7753432: 2 (-2)
 ```
Note that the first price has no associated change because there was no previous price to compare it with.

In this short example, within just these first few prices, the highest price will be 6, so it would be nice to give the monkey instructions that would make it sell at that time. The first 6 occurs after only two changes, so there's no way to instruct the monkey to sell then, but the second 6 occurs after the changes -1,-1,0,2. So, if you gave the monkey that sequence of changes, it would wait until the first time it sees that sequence and then immediately sell your hiding spot information at the current price, winning you 6 bananas.

Each buyer only wants to buy one hiding spot, so after the hiding spot is sold, the monkey will move on to the next buyer. If the monkey never hears that sequence of price changes from a buyer, the monkey will never sell, and will instead just move on to the next buyer.

Worse, you can only give the monkey a single sequence of four price changes to look for. You can't change the sequence between buyers.

You're going to need as many bananas as possible, so you'll need to determine which sequence of four price changes will cause the monkey to get you the most bananas overall. Each buyer is going to generate 2000 secret numbers after their initial secret number, so, for each buyer, you'll have 2000 price changes in which your sequence can occur.

Suppose the initial secret number of each buyer is:
```
1
2
3
2024
```
There are many sequences of four price changes you could tell the monkey, but for these four buyers, the sequence that will get you the most bananas is -2,1,-1,3. Using that sequence, the monkey will make the following sales:

For the buyer with an initial secret number of 1, changes -2,1,-1,3 first occur when the price is 7.
For the buyer with initial secret 2, changes -2,1,-1,3 first occur when the price is 7.
For the buyer with initial secret 3, the change sequence -2,1,-1,3 does not occur in the first 2000 changes.
For the buyer starting with 2024, changes -2,1,-1,3 first occur when the price is 9.
So, by asking the monkey to sell the first time each buyer's prices go down 2, then up 1, then down 1, then up 3, you would get 23 (7 + 7 + 9) bananas!

Figure out the best sequence to tell the monkey so that by looking for that same sequence of changes in every buyer's future prices, you get the most bananas in total. What is the most bananas you can get?

Summarize the part two for me.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠

Part 2 changes the goal:

1. **Prices** are now the last digit of each secret number (e.g., 123 → price 3)
2. A monkey will sell when it sees a specific sequence of **4 consecutive price changes**
3. You pick **one sequence** that applies to all buyers
4. For each buyer, the monkey sells at the **first occurrence** of that sequence (getting that price in bananas)
5. Goal: Find the 4-change sequence that maximizes total bananas across all buyers

For the sample `[1, 2, 3, 2024]`, the best sequence is `(-2, 1, -1, 3)` yielding 23 bananas.

Want to start by generating prices and changes for a single buyer?

How can I write price_gen to reuse calc_gen?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

You can use a generator expression or `yield from` with a transformation:

```python
def price_gen(secret):
    for s in calc_gen(secret):
        yield s % 10
```

Or more concisely:

```python
def price_gen(secret):
    return (s % 10 for s in calc_gen(secret))
```

In [ ]:
def price_gen(secret):
    for s in calc_gen(secret):
        yield s % 10
list(islice(price_gen(123), 10))

[0, 6, 5, 4, 4, 6, 4, 4, 2, 4]

In [ ]:
def price_gen(secret):
    return (s % 10 for s in calc_gen(secret))
list(islice(price_gen(123), 10))

[0, 6, 5, 4, 4, 6, 4, 4, 2, 4]

I understand the version with `yield`. But how exactly the second version work?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠

The second version returns a **generator expression** — it's like a list comprehension but with parentheses instead of brackets. It creates a generator object lazily (values computed on demand, not all at once).

Key difference:
- `[s % 10 for s in calc_gen(secret)]` — list comprehension, computes everything immediately
- `(s % 10 for s in calc_gen(secret))` — generator expression, computes values one at a time when requested

Since `calc_gen` is infinite, a list comprehension would hang forever. The generator expression works because it only pulls values as needed (e.g., when `islice` asks for them).

Does that clarify it?

The `price_gen` should return secret % 10 as a first value.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Good catch! You need to yield the initial price first before the generated ones:

```python
def price_gen(secret):
    yield secret % 10
    for s in calc_gen(secret):
        yield s % 10
```

In [ ]:
def price_gen(secret):
    yield secret % 10
    for s in calc_gen(secret):
        yield s % 10
list(islice(price_gen(123), 10))

[3, 0, 6, 5, 4, 4, 6, 4, 4, 2]

Can this be rewritten using generator expression?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Not cleanly — generator expressions can't have a "prefix" value before the iteration starts. You'd need `itertools.chain`:

```python
from itertools import chain

def price_gen(secret):
    return chain([secret % 10], (s % 10 for s in calc_gen(secret)))
```

But the `yield` version is more readable here.

What do we need next?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Next you'll want to compute the **changes** (differences between consecutive prices). Then you can look for 4-change sequences.

Have you used `itertools.pairwise` before? It's handy for getting consecutive pairs.

In [ ]:
from itertools import *

def changes_gen(secret):
    for a, b in pairwise(price_gen(secret)):
        yield b - a
L(islice(changes_gen(123), 100))

[-3, 6, -1, -1, 0, 2, -2, 0, -2, 2, -4, 4, -4, 3, 6, -8, 3, 1, 1, 0, -3, 1, 2, -1, 1, 3, -5, 2, -2, 3, -3, 3, -1, -6, 8, -1, -2, 4, -6, -1, 5, -7, 2, -2, 3, -3, 5, -4, 5, -3, 3, 1, 2, -6, 6, -7, -2, 5, 3, 1, -6, -1, 5, -3, 5, -4, 0, -3, 3, 1, 0, 2, -8, 0, 8, -3, -2, 4, -2, 1, 2, -4, 1, 4, -6, -3, 2, 0, 1, 6, -1, -1, -1, -2, 3, -3, -4, 3, -2, -1]

Ok, what's next?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Now you need to pair each 4-change sequence with the price you'd get. For each buyer, you want to find the **first occurrence** of each 4-tuple and record the price at that point.

Have you used sliding windows before? You could use `collections.deque(maxlen=4)` or check out `itertools` recipes for a `sliding_window` function.

In [ ]:
!pip install more-itertools

In [ ]:
from more_itertools import *

In [ ]:
ws_gen = sliding_window(changes_gen(123), 4)
L(islice(ws_gen, 100))

[(-3, 6, -1, -1), (6, -1, -1, 0), (-1, -1, 0, 2), (-1, 0, 2, -2), (0, 2, -2, 0), (2, -2, 0, -2), (-2, 0, -2, 2), (0, -2, 2, -4), (-2, 2, -4, 4), (2, -4, 4, -4), (-4, 4, -4, 3), (4, -4, 3, 6), (-4, 3, 6, -8), (3, 6, -8, 3), (6, -8, 3, 1), (-8, 3, 1, 1), (3, 1, 1, 0), (1, 1, 0, -3), (1, 0, -3, 1), (0, -3, 1, 2), (-3, 1, 2, -1), (1, 2, -1, 1), (2, -1, 1, 3), (-1, 1, 3, -5), (1, 3, -5, 2), (3, -5, 2, -2), (-5, 2, -2, 3), (2, -2, 3, -3), (-2, 3, -3, 3), (3, -3, 3, -1), (-3, 3, -1, -6), (3, -1, -6, 8), (-1, -6, 8, -1), (-6, 8, -1, -2), (8, -1, -2, 4), (-1, -2, 4, -6), (-2, 4, -6, -1), (4, -6, -1, 5), (-6, -1, 5, -7), (-1, 5, -7, 2), (5, -7, 2, -2), (-7, 2, -2, 3), (2, -2, 3, -3), (-2, 3, -3, 5), (3, -3, 5, -4), (-3, 5, -4, 5), (5, -4, 5, -3), (-4, 5, -3, 3), (5, -3, 3, 1), (-3, 3, 1, 2), (3, 1, 2, -6), (1, 2, -6, 6), (2, -6, 6, -7), (-6, 6, -7, -2), (6, -7, -2, 5), (-7, -2, 5, 3), (-2, 5, 3, 1), (5, 3, 1, -6), (3, 1, -6, -1), (1, -6, -1, 5), (-6, -1, 5, -3), (-1, 5, -3, 5), (5, -3, 5, -4), (

What's next?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Now you need to pair each 4-change window with the **price at that position**. The price corresponds to the 5th element in the sequence (after 4 changes).

One approach: zip the sliding windows with the prices (offset by 4). How would you like to tackle that?

How can I zip the generators?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

You need to call the functions to create the generators:

```python
ws_gen = sliding_window(changes_gen(123), 4)
prices = islice(price_gen(123), 4, None)  # skip first 4 prices
zipped_gen = zip(prices, ws_gen)
```

The price after 4 changes is at index 4, so we skip the first 4 prices.

In [ ]:
def price_with_changes_gen(secret):
    ws_gen = sliding_window(changes_gen(secret), 4)
    prices = islice(price_gen(secret), 4, None)  # skip first 4 prices
    return zip(ws_gen, prices)

L(islice(price_with_changes_gen(123), 10))

[((-3, 6, -1, -1), 4), ((6, -1, -1, 0), 4), ((-1, -1, 0, 2), 6), ((-1, 0, 2, -2), 4), ((0, 2, -2, 0), 4), ((2, -2, 0, -2), 2), ((-2, 0, -2, 2), 4), ((0, -2, 2, -4), 0), ((-2, 2, -4, 4), 4), ((2, -4, 4, -4), 0)]

Does the above function look correct?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Yes, looks correct! Checking against the problem example for secret 123:

- Prices: 3, 0, 6, 5, 4, 4, 6, 4, 4, 2...
- First window `(-3, 6, -1, -1)` → price 4 (at index 4) ✓
- Window `(-1, -1, 0, 2)` → price 6 ✓

The tuple order is `(changes, price)` which works fine. Now you need to build a dict for each buyer mapping each sequence to its **first** occurrence price.

Can I use `unique` function to keep only the first occurances?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Yes! `more_itertools.unique_everseen` with a `key` function would work:

```python
unique_everseen(price_with_changes_gen(123), key=lambda x: x[1])
```

This keeps only the first `(price, changes)` pair for each unique `changes` tuple.

In [ ]:
L(unique_everseen(islice(price_with_changes_gen(123), 10), first))

[((-3, 6, -1, -1), 4), ((6, -1, -1, 0), 4), ((-1, -1, 0, 2), 6), ((-1, 0, 2, -2), 4), ((0, 2, -2, 0), 4), ((2, -2, 0, -2), 2), ((-2, 0, -2, 2), 4), ((0, -2, 2, -4), 0), ((-2, 2, -4, 4), 4), ((2, -4, 4, -4), 0)]

In [ ]:
dict(unique_everseen(islice(price_with_changes_gen(123), 10), first))

{(-3, 6, -1, -1): 4,
 (6, -1, -1, 0): 4,
 (-1, -1, 0, 2): 6,
 (-1, 0, 2, -2): 4,
 (0, 2, -2, 0): 4,
 (2, -2, 0, -2): 2,
 (-2, 0, -2, 2): 4,
 (0, -2, 2, -4): 0,
 (-2, 2, -4, 4): 4,
 (2, -4, 4, -4): 0}

Will the above work?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Yes, that approach works! The dict will map each change sequence to its first occurrence price.

For the full solution, you'll want to remove the `islice(..., 10)` and instead get all 2000 iterations. Something like `islice(price_with_changes_gen(secret), 2000-3)` (since you need 2000 secrets but lose 4 for the first window).

In [ ]:
def get_counts(secret):
    return dict(unique_everseen(islice(price_with_changes_gen(secret), 2000-3), first))
get_counts(1).get((-2,1,-1,3))

7

In [ ]:
L(1,2,3,2024).map(get_counts).map(lambda d: d.get((-2,1,-1,3), 0))

[7, 7, 0, 9]

What's next?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Now you need to find which sequence gives the **maximum total** across all buyers. You'll need to:

1. Get the counts dict for each buyer
2. Sum the values for each sequence across all buyers
3. Find the max

A `Counter` could help here — you can add Counters together to sum matching keys.

In [ ]:
L(1,2,3,2024).map(get_counts)

[{(2, 0, 6, -4): 5, (0, 6, -4, 4): 9, (6, -4, 4, -9): 0, (-4, 4, -9, 5): 5, (4, -9, 5, 4): 9, (-9, 5, 4, -2): 7, (5, 4, -2, -5): 2, (4, -2, -5, -2): 0, (-2, -5, -2, 1): 1, (-5, -2, 1, 2): 3, (-2, 1, 2, 6): 9, (1, 2, 6, -1): 8, (2, 6, -1, -3): 5, (6, -1, -3, 2): 7, (-1, -3, 2, -1): 6, (-3, 2, -1, 0): 6, (2, -1, 0, 3): 9, (-1, 0, 3, -6): 3, (0, 3, -6, 4): 7, (3, -6, 4, -2): 5, (-6, 4, -2, -3): 2, (4, -2, -3, -1): 1, (-2, -3, -1, 2): 3, (-3, -1, 2, 5): 8, (-1, 2, 5, -3): 5, (2, 5, -3, -2): 3, (5, -3, -2, 2): 5, (-3, -2, 2, -3): 2, (-2, 2, -3, 3): 5, (2, -3, 3, 3): 8, (-3, 3, 3, 0): 8, (3, 3, 0, -1): 7, (3, 0, -1, -3): 4, (0, -1, -3, -4): 0, (-1, -3, -4, 7): 7, (-3, -4, 7, -4): 3, (-4, 7, -4, 2): 5, (7, -4, 2, 1): 6, (-4, 2, 1, -3): 3, (2, 1, -3, 4): 7, (1, -3, 4, -7): 0, (-3, 4, -7, 1): 1, (4, -7, 1, 4): 5, (-7, 1, 4, 0): 5, (1, 4, 0, 3): 8, (4, 0, 3, -2): 6, (0, 3, -2, 2): 8, (3, -2, 2, 1): 9, (-2, 2, 1, -5): 4, (2, 1, -5, 3): 7, (1, -5, 3, 0): 7, (-5, 3, 0, 2): 9, (3, 0, 2, -8): 1, (0, 

In [ ]:
from collections import Counter

L(1,2,3,2024).map(get_counts).map(Counter)

[Counter({(0, 6, -4, 4): 9, (4, -9, 5, 4): 9, (-2, 1, 2, 6): 9, (2, -1, 0, 3): 9, (3, -2, 2, 1): 9, (-5, 3, 0, 2): 9, (-3, 7, -4, 5): 9, (-3, -3, -1, 9): 9, (-1, 9, -1, 1): 9, (-1, 0, -6, 9): 9, (9, -5, 0, 5): 9, (0, -8, 5, 4): 9, (-3, 0, -3, 6): 9, (2, -3, -4, 8): 9, (0, -1, -1, 8): 9, (0, 1, 4, 3): 9, (-6, -2, 4, 5): 9, (-9, 3, 0, 6): 9, (-1, 4, -3, 7): 9, (-3, 7, -6, 6): 9, (4, 2, -3, 4): 9, (6, -2, -6, 9): 9, (0, 0, -5, 7): 9, (0, -5, 7, 0): 9, (7, 0, -2, 2): 9, (-7, 3, 2, 2): 9, (3, 2, 2, 0): 9, (-4, 0, 1, 7): 9, (0, -4, 7, 1): 9, (6, 1, -6, 8): 9, (8, -5, 2, 3): 9, (1, -3, -2, 7): 9, (0, -5, 2, 5): 9, (3, -6, 6, 3): 9, (6, -5, -1, 8): 9, (-1, 8, -2, 2): 9, (-2, 3, -3, 7): 9, (-3, 7, -9, 9): 9, (1, 1, 3, 1): 9, (0, -3, -4, 8): 9, (2, 1, -1, 7): 9, (-1, 1, -1, 5): 9, (8, -7, 5, 3): 9, (-2, 6, -4, 5): 9, (-4, 5, -9, 9): 9, (5, -1, -1, 5): 9, (-1, 5, -6, 6): 9, (-7, 4, 1, 3): 9, (4, -7, 5, 4): 9, (1, -5, 2, 4): 9, (1, 0, -5, 8): 9, (-2, 0, -2, 8): 9, (3, -1, 5, 2): 9, (-1, 5, 2, 0): 

In [ ]:
from operator import add

L(1,2,3,2024).map(get_counts).map(Counter).reduce(add)

Counter({(-2, 1, -1, 3): 23,
         (-1, 3, 1, 0): 22,
         (1, -3, 5, 1): 21,
         (0, 5, 0, 0): 20,
         (-5, 6, 0, -1): 20,
         (-4, -1, 5, -1): 20,
         (-1, 3, 2, 0): 19,
         (0, 5, 0, -1): 19,
         (-1, 2, 0, 3): 19,
         (-3, 7, -6, 6): 18,
         (0, 0, -5, 7): 18,
         (-7, 2, 5, 1): 18,
         (-4, 1, 5, 3): 18,
         (5, -1, 3, 0): 18,
         (2, 6, -8, 8): 18,
         (6, -8, 8, 0): 18,
         (-2, -4, 5, 3): 18,
         (-6, 1, 8, 0): 18,
         (3, -6, -1, 7): 18,
         (-4, 1, 1, 2): 18,
         (6, 2, -6, 7): 18,
         (2, -1, 3, 4): 18,
         (-5, 0, 9, 0): 18,
         (7, -6, 5, 2): 18,
         (-6, 5, -4, 5): 18,
         (-2, -3, 3, 4): 18,
         (-2, 6, -6, 8): 18,
         (-5, 4, 2, 0): 18,
         (5, -5, -3, 8): 18,
         (8, -9, 1, 8): 18,
         (-2, -4, 6, 3): 18,
         (-1, -2, 0, 4): 18,
         (0, -8, 4, 5): 18,
         (0, 2, 1, 0): 18,
         (5, -1, 2, 2): 18,
         

In [ ]:
L(1,2,3,2024).map(get_counts).map(Counter).reduce(add).most_common(1)

[((-2, 1, -1, 3), 23)]

In [ ]:
L(1,2,3,2024).map(get_counts).map(Counter).reduce(add).most_common(1)[0][1]

23

In [ ]:
def solve(inp):
    xs = L(inp.splitlines()).map(int)
    all_counter = xs.map(get_counts).map(Counter).reduce(add)
    return all_counter.most_common(1)[0][1]
# %time solve(inp) # Takes more than 5 minutes

# Reflect

The solution is slow - let's improve that.

In [ ]:
%timeit get_counts(123);

3.38 ms ± 6.35 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## JIT with Numba

In [ ]:
from numba import njit

@njit
def prune(num): return num % 16777216

@njit
def mix(secret, num): return secret ^ num

@njit
def calc(secret):
    secret = prune(mix(secret, secret * 64))
    secret = prune(mix(secret, secret // 32))
    secret = prune(mix(secret, secret * 2048))
    return secret
calc(123)

15887950

In [ ]:
%timeit get_counts(123);

2.01 ms ± 7.29 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Removing generators

The `solve` function is slow. Let's refactor the solution and get rid of the generators. Keep `calc` function.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Let's compute all 2000 secrets directly in a list, then derive prices and changes from that:

```python
def get_secrets(secret, n=2000):
    res = [secret]
    for _ in range(n): res.append(calc(res[-1]))
    return res

def get_counts_fast(secret):
    secrets = get_secrets(secret)
    prices = [s % 10 for s in secrets]
    changes = [prices[i+1] - prices[i] for i in range(len(prices)-1)]
    seen = {}
    for i in range(len(changes)-3):
        key = tuple(changes[i:i+4])
        if key not in seen: seen[key] = prices[i+4]
    return seen
```

In [ ]:
@njit
def get_secrets(secret, n=2000):
    res = [secret]
    for _ in range(n): res.append(calc(res[-1]))
    return res

def get_counts(secret):
    secrets = get_secrets(secret)
    prices = [s % 10 for s in secrets]
    changes = [prices[i+1] - prices[i] for i in range(len(prices)-1)]
    seen = {}
    for i in range(len(changes)-3):
        key = tuple(changes[i:i+4])
        if key not in seen: seen[key] = prices[i+4]
    return seen

In [ ]:
%timeit get_counts(123);

852 μs ± 2.82 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Can I apply both @cache and @njit to the `calc` function?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠

No, they don't work together. `@njit` compiles to machine code and doesn't support Python decorators like `@cache`.

For numba, you'd use a manual cache dict outside the function, or just rely on numba's speed — at this point `get_secrets` is already fast since it's njit-compiled.

If you want caching for `get_secrets`, numba has a `cache=True` option:

```python
@njit(cache=True)
def get_secrets(secret, n=2000):
    ...
```

This caches the compiled function to disk (not the results).

I want to optimize the functions further and get rid of unnecessary lists.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

```python
def get_counts_fast(secret):
    seen = {}
    prices = [secret % 10]
    for _ in range(2000):
        secret = calc(secret)
        prices.append(secret % 10)
        if len(prices) >= 5:
            key = (prices[-4]-prices[-5], prices[-3]-prices[-4], prices[-2]-prices[-3], prices[-1]-prices[-2])
            if key not in seen: seen[key] = prices[-1]
    return seen

get_counts_fast(1).get((-2,1,-1,3))
```

In [ ]:
@njit
def get_counts(secret):
    seen = {}
    prices = [secret % 10]
    for _ in range(2000):
        secret = calc(secret)
        prices.append(secret % 10)
        if len(prices) >= 5:
            key = (prices[-4]-prices[-5], prices[-3]-prices[-4], prices[-2]-prices[-3], prices[-1]-prices[-2])
            if key not in seen: seen[key] = prices[-1]
    return seen

%timeit get_counts(123);

235 μs ± 62.1 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%time solve(inp)

CPU times: user 1min 17s, sys: 980 ms, total: 1min 18s
Wall time: 1min 18s


2246

## Removing merging of large dicts

How can I speed up the `solve` function further?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

The bottleneck is likely the `Counter.reduce(add)` step — merging many large dicts is slow. Try accumulating into a single dict directly:

```python
from collections import defaultdict

def solve(inp):
    xs = L(inp.splitlines()).map(int)
    totals = defaultdict(int)
    for x in xs:
        for k, v in get_counts(x).items(): totals[k] += v
    return max(totals.values())

%time solve(inp)
```

In [ ]:
from collections import defaultdict

def solve(inp):
    xs = L(inp.splitlines()).map(int)
    totals = defaultdict(int)
    for x in xs:
        for k, v in get_counts(x).items(): totals[k] += v
    return max(totals.values())

%time solve(inp)

CPU times: user 14.2 s, sys: 6.6 ms, total: 14.2 s
Wall time: 14.2 s


2246

## Parallelization (doesn't work with njit)

Anything else I can do to speed up the `solve` function? What about parallelizing the get_counts? Let's use `parallel` from fastcore.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠

```python
from fastcore.parallel import parallel

def solve(inp):
    xs = L(inp.splitlines()).map(int)
    counts = parallel(get_counts, xs, progress=True)
    totals = defaultdict(int)
    for c in counts:
        for k, v in c.items(): totals[k] += v
    return max(totals.values())

%time solve(inp)
```

In [ ]:
from fastcore.parallel import parallel

def solve(inp):
    xs = L(inp.splitlines()).map(int)
    counts = parallel(get_counts, xs, progress=True)
    totals = defaultdict(int)
    for c in counts:
        for k, v in c.items(): totals[k] += v
    return max(totals.values())

%time solve(inp)

CPU times: user 191 ms, sys: 316 ms, total: 507 ms
Wall time: 1.43 s


TypeError: cannot pickle '_nrt_python._MemInfo' object

How can I solve the error above?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

The `@njit` decorated function returns objects that can't be pickled for multiprocessing. Create a plain Python wrapper:

```python
def get_counts_wrapper(secret):
    return dict(get_counts(secret))

def solve(inp):
    xs = L(inp.splitlines()).map(int)
    counts = parallel(get_counts_wrapper, xs, progress=True)
    totals = defaultdict(int)
    for c in counts:
        for k, v in c.items(): totals[k] += v
    return max(totals.values())

%time solve(inp)
```